# Aufgabenstellung

Du arbeitest als Data Scientist bei einer P-2-P-Plattform, die vor einem Jahr gegründet wurde. Nun wollt ihr euer Geschäft erweitern. Euer Team hat sich aufgeteilt und jeder Analyst hat einen Teilbereich der Daten. Deine Aufgabe ist es in einer *explorativen Datenanalyse* Insights für eure Plattform herauszufinden.

Euer Geschäftsmodell ist das Betreiben einer Plattform (crowd-investing) bei der sich Personen die eine Geschäftsidee haben, aber nicht das benötigte Geld, anmelden und für ihr Projekt innerhalb einer vorgegebenen Zeit Geld sammeln können. Auf der anderen Seite habt ihr Geldgeber, die gern ihr Geld in Projekte anlegen möchten und nach Investitionen suchen. Als Vermittler bringt eure Plattform also Geldnehmer und Geldgeber zusammen. Ihr verdient euer Geld mit einer Provision für jedes Projekt was auf eurer Plattform landet.

Deine Datenbasis ist die Historie eurer Plattform. Alle Projekte sind abgeschlossene Projekte, d.h. die Zeit, um für sein Projekt Geld zu sammeln ist abgelaufen. Euer Geschäftsmodell sieht es vor, dass die gesammelten Gelder ausgezahlt werden, auch wenn der Zielbetrag nicht erreicht wurde.

Dafür ist der Datensatz zunächst vorzubereiten. In die Bewertung geht zudem die Dokumentation mit ein. Für die Visualisierung sind 3 verschiedenartige Plots zu benennen, die in die Bewertung eingehen sollen. Das bedeutet nicht, dass du nur 3 Plots ersellen darfst - ausschließlich für die Bewertung sind drei verschiedenartige Plots zu benennen! Vergiss dabei nicht die Erkenntnisse die aus jedem Plot gezogen werden können, zu notieren.

Die Dokumentation kann in englisch oder deutsch erfolgen.

Am Freitag erfolgt die Vorstellung der Plots innerhalb des Zeitrahmens von 5min - (+-1min) ist dabei ok.

Treff ist **Freitag um 14:30 Uhr im Hauptraum**. Da werden wir die Unterlagen in dokumentensichere Formate umwandeln. Das machen wir aber zusammen. Danach habt ihr bis 15:35 Uhr zeit, eure zu bewertenden Unterlagen in den Projektabgabeordner hochzuladen.


## Inhalt des Datensatzes
Der gesplittete Datensatz enthält folgende Spalten (inkl. Bedeutung):

- funded_amount ... mit Ablauf der "Crowding"zeit erhaltener Betrag/ ausgezahlter Betrag in USD
- loan_amount ... Zielbetrag (Betrag dem man für das Projekt erreichen wollte) in USD
- activity ... Unterkategory zu dem das Ziel des Crowdprojektes thematisch gehört
- sector ... Oberkategory in den das Crowdprojektes Thema fällt
- use ... Kurzbeschreibung wofür das Geld verwendet werden soll
- country_code ... Ländercode nach ISO Norm
- country ... Ländername nach ISO Norm
- region ... Region
- currency ... Währung in den der funded_amount dann ausgezahlt wurde
- term in months ... Dauer über die der Kredit ausgezahlt werden soll
- lender_count ...Darlehensgeber (also wieviele Personen Geld für das Projekt gegeben haben)
- borrower_genders ... Geschlecht und Anzahl der Darlehensnehmer, also diejenigen die das Crowdprojekt initiiert haben     
- repayment interval ... vertraglich vereinbarte Rückzahlungsmodalitäten/-rhythmus

# Projektbearbeitung - Crowd-Investing

## Vorbereitung - Importieren der Bibliotheken und Einlesen des Datensatzes

Bevor die eigentliche Arbeit beginnen kann, müssen zuvor einige notwendige Bibliotheken eingelesen werden. Danach wird der Datensatz eingelesen, welcher im vorliegenden Fall aus zwei Dateien besteht. Die Daten liegen in einem CSV-Format vor. Hier muss beachtet werden, was als Trennzeichen in der jeweiligen Datei benutzt wurde. Dies muss beim Import berücksichtigt werden. 

In [1]:
import pandas as pd
import numpy as np

from currency_converter import CurrencyConverter
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

from datetime import date
from colorama import Fore, Back

In [2]:
file1 = 'your_part1.csv'
file2 = 'your_part2.csv'

In [3]:
df1 = pd.read_csv(file1, index_col=0)
df1.head()

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
0,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,12.0,12,female,irregular
1,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,11.0,14,"female, female",irregular
2,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,43.0,6,female,bullet
3,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,11.0,8,female,irregular
4,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,14.0,16,female,monthly


In [4]:
df1.shape

(335000, 13)

In [5]:
df2 = pd.read_csv(file2, usecols=[1])
df2.head(1)

,# funded_amount# loan_amount# activity# sector# use# country_code# country# region# currency# term_in_months# lender_count# borrower_genders# repayment_interval
0,Leyte#PHP#8.0#6#female#irregular


Beim Import der Datei gab es einen Fehler. Das Einlesen der ersten Spalte klappte hingegen und wie man sehen kann, muss das Trennzeichen (hier auf #) angepasst werden.

In [6]:
df2 = pd.read_csv(file2,
                  delimiter='#', # Trennzeichen muss angepasst werden
                  index_col=0
                 ) 
df2.head()

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
0,175.0,175.0,Liquor Store / Off-License,Food,to purchase additional stock of coconut wine t...,PH,Philippines,"Palo, Leyte",PHP,8.0,6,female,irregular
1,325.0,325.0,Livestock,Agriculture,to buy 3 zebus and food to fatten them up.,MG,Madagascar,Antsirabe,MGA,12.0,13,female,monthly
2,550.0,550.0,Food Stall,Food,to buy ingredients for her food-vending busine...,PH,Philippines,"Cordova, Cebu",PHP,5.0,6,female,irregular
3,1300.0,1300.0,Cattle,Agriculture,to buy one head of cattle.,EG,Egypt,Baniswef,EGP,14.0,50,male,monthly
4,900.0,900.0,Consumer Goods,Personal Use,to buy consumer goods amongst others.,PE,Peru,Urubamba - Urubamba - Cusco,PEN,6.0,1,female,irregular


In [7]:
df2.shape

(336205, 13)

### Zusammenfügen der Datensätze

Nachdem nun beide Datensätze eingelesen wurden, konnten diese miteinander verglichen werden. Beide Datensätze bestehen aus der selben Anzahl an Spalten (hier sind es 13 an der Zahl) und können somit zeilenweise verbunden werden (d.h. es gibt keine neuen Eigenschaften im zweiten Datensatz und die Daten können einfach unten angehangen werden)

In [8]:
df = pd.concat([df1, df2], axis=0)
df

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
0,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,12.0,12,female,irregular
1,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,11.0,14,"female, female",irregular
2,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,43.0,6,female,bullet
3,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,11.0,8,female,irregular
4,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,14.0,16,female,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...
336200,0.0,25.0,Livestock,Agriculture,"[True, u'para compara: cemento, arenya y ladri...",PY,Paraguay,Concepción,USD,13.0,0,female,monthly
336201,25.0,25.0,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",KE,Kenya,NaN,KES,13.0,1,female,monthly
336202,0.0,25.0,Games,Entertainment,NaN,KE,Kenya,NaN,KES,13.0,0,NaN,monthly
336203,0.0,25.0,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",KE,Kenya,NaN,KES,13.0,0,female,monthly


### Prüfung des Resultates

Es wird geprüft, ob das Zusammenfügen der Dateien geklappt hat. Wir kennen die Dimensionen (Anzahl der Zeilen/Spalten) der ersten beiden Datensätze und können dies mit dem kompletten Datensatz vergleichen. Da wir die Daten als neue Zeilen unter den ersten angefügt haben, darf sich die Anzahl der Spalten nicht verändert haben, die Zeilen jedoch sollten der Summe aus Datensatz1 und Datensatz2 entsprechen.

In [9]:
if df1.shape[1] == df2.shape[1] & df1.shape[1] == df.shape[1]:
    print('Alle Datensätze haben die gleiche Spaltenanzahl!\n')

if df.shape[0] == df1.shape[0] + df2.shape[0]:
    print('Zeilenweises Zusammenfügen hat funktioniert!\n')
    print(
        'Der komplette Datensatz besteht aus {rows} Zeilen und {columns} Spalten'
        .format(rows=df.shape[0], columns=df.shape[1]))

Alle Datensätze haben die gleiche Spaltenanzahl!

Zeilenweises Zusammenfügen hat funktioniert!

Der komplette Datensatz besteht aus 671205 Zeilen und 13 Spalten


Das Zusammenfügen hat funktioniert. Nun sotiere ich den Datensatz noch nach dem Land und setze den Index entsprechend zurück. Anschließend können wir uns den Datensatz etwas genauer anschauen.

## Preprocessing des Datensatzes

Nachfolgend wird der Datensatz nach folgenden Gesichtspunkten geprüft:

- erste grobe statistische Übersicht anzeigen
- unnötige Leerzeichen bei Zeichenketten (um später die daten besser selektieren zu können)
- passen die Datentypen zu dem Inhalt der Spalten
- fehlende Werte (sowohl reale als auch synonymisierte Werte)
- können Daten kategorisiert werden (z.B. einige wenige Werte für ganz viele Zeilen)

### Einfache statistische Übersicht

In [10]:
df.describe()

,funded_amount,loan_amount,term_in_months,lender_count
count,671205.000000,671205.000000,671205.000000,671205.000000
mean,785.995061,842.397107,13.739022,20.590922
std,1130.398941,1198.660073,8.598919,28.459551
min,0.000000,25.000000,1.000000,0.000000
25%,250.000000,275.000000,8.000000,7.000000
50%,450.000000,500.000000,13.000000,13.000000
75%,900.000000,1000.000000,14.000000,24.000000
max,100000.000000,100000.000000,158.000000,2986.000000


Hier fällt auf, das wir in den Spalten **funded_amount** und **lender_count** als Minimalwert '0' angegeben haben. Das könnte ein ersten Hinweis auf fehlende Werte sein. Vielleicht hängen beide Werte voneinander ab, denn wo kein Dahrlehensgeber ist, da kann auch kein Geld geflossen sein. Aber dies wird zu einem späteren Zeitpunkt überprüft.

In [11]:
df.nunique()

 funded_amount            610
 loan_amount              479
 activity                 163
 sector                    15
 use                   424912
 country_code              86
 country                   87
 region                 12695
 currency                  67
 term_in_months           148
 lender_count             503
 borrower_genders       11298
 repayment_interval         4
dtype: int64

Hier fällt der besonders hohe Anteil der einzigartigen Werte in der Spalte **borrower_genders** auf. Beim Geschlecht geht man erstmal von einer geringeren Anzahl aus (nach heutigem Standard sind es drei), doch hier scheint man davon abzuweichen. Wir werden später prüfen, was es damit aufsich hat.

### Leerzeichen in Spaltenüberschriften

Es wird geprüft, ob es Leerzeichen in den Spaltenbeschriftungen gibt, welche das spätere selektieren/filtern der Daten beeinträchtigen kann. Wenn vorhanden, werden diese schließlich entfernt.

In [12]:
print('Überschriften vorher:')
df.columns

Überschriften vorher:


Index([' funded_amount', ' loan_amount', ' activity', ' sector', ' use',
       ' country_code', ' country', ' region', ' currency', ' term_in_months',
       ' lender_count', ' borrower_genders', ' repayment_interval'],
      dtype='object')

In [13]:
for column in df.columns:
    if column[0].isspace():
        df.rename(columns=lambda column: column.strip(), inplace=True)

In [14]:
#check, ob Verarbeitung geklappt hat

print('Überschriften nacher:')
df.columns

Überschriften nacher:


Index(['funded_amount', 'loan_amount', 'activity', 'sector', 'use',
       'country_code', 'country', 'region', 'currency', 'term_in_months',
       'lender_count', 'borrower_genders', 'repayment_interval'],
      dtype='object')

### Sortieren und neu indizieren

In [15]:
df.sort_values(by=['country'], inplace=True)

In [16]:
df.reset_index(drop=True, inplace=True)
df

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
0,8000.0,8000.0,Textiles,Arts,"To purchase raw materials, including fabrics, ...",AF,Afghanistan,Kandahar City,USD,6.0,308,female,bullet
1,6000.0,6000.0,Textiles,Arts,To buy 9 sewing machines that will increase pr...,AF,Afghanistan,Kandahar City,USD,8.0,204,female,bullet
2,825.0,825.0,Agriculture,Agriculture,to buy a dairy cow,AL,Albania,Korce,ALL,20.0,26,female,monthly
3,2775.0,2775.0,Agriculture,Agriculture,to repair the damaged animal stable and also d...,AL,Albania,Korce,ALL,26.0,101,male,monthly
4,825.0,825.0,Personal Housing Expenses,Housing,to repair windows and doors in her house.,AL,Albania,Korce,ALL,14.0,32,female,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...
671200,1900.0,1900.0,Poultry,Agriculture,To expand her production of chickens.,ZW,Zimbabwe,Harare,USD,8.0,69,"female, female, female, female",monthly
671201,1500.0,1500.0,Clothing,Clothing,to buy clothes and school uniforms for resale.,ZW,Zimbabwe,GOKWE,USD,8.0,36,female,monthly
671202,500.0,500.0,Clothing Sales,Clothing,To purchase more clothes to sell,ZW,Zimbabwe,Hurungwe,USD,8.0,19,female,irregular
671203,1100.0,1100.0,Grocery Store,Food,to restock a wider range of stock for her tabl...,ZW,Zimbabwe,Harare,USD,9.0,30,"female, female, female, female",irregular


In [17]:
# Funktion zur Ausgabe des genutzten Speicherplatzes in MiB
def my_memory_usage(dataframe):
    # Umrechnungsfaktor Byte --> MiB
    MiB = 2**(20)
    
    mem = dataframe.memory_usage(deep=True)
    n = round(mem.sum() / MiB, 1)
    print('genutzer Speicherplatz [MiB]:')
    return n

In [18]:
mem_usage_init = my_memory_usage(df)
mem_usage_init

genutzer Speicherplatz [MiB]:


429.9

In [19]:
df.info(verbose=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671205 entries, 0 to 671204
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   funded_amount       671205 non-null  float64
 1   loan_amount         671205 non-null  float64
 2   activity            671205 non-null  object 
 3   sector              671205 non-null  object 
 4   use                 666973 non-null  object 
 5   country_code        671197 non-null  object 
 6   country             671205 non-null  object 
 7   region              614405 non-null  object 
 8   currency            671205 non-null  object 
 9   term_in_months      671205 non-null  float64
 10  lender_count        671205 non-null  int64  
 11  borrower_genders    666984 non-null  object 
 12  repayment_interval  671205 non-null  object 
dtypes: float64(3), int64(1), object(9)
memory usage: 429.9 MB


### Prüfung auf fehlende Werte

Nun prüfen wir den Datensatz auf fehlende Werte. Anschlißend wird bewertet, was wir mit diesen Daten machen (Löschen, Konvertieren, berechnete Werte eintragen).

In [20]:
null_values = df.isnull().sum()
null_values

funded_amount             0
loan_amount               0
activity                  0
sector                    0
use                    4232
country_code              8
country                   0
region                56800
currency                  0
term_in_months            0
lender_count              0
borrower_genders       4221
repayment_interval        0
dtype: int64

In [21]:
cols_with_nulls = null_values[null_values > 0] 
cols_with_nulls

use                  4232
country_code            8
region              56800
borrower_genders     4221
dtype: int64

In [22]:
cols_with_nulls.index

Index(['use', 'country_code', 'region', 'borrower_genders'], dtype='object')

In [23]:
n_rows = df.shape[0]

for i, column in enumerate(cols_with_nulls.index):
    name = 'Anzahl der fehlenden Werte in Spalte "{0}":'.format(column)
    missing = cols_with_nulls.values[i]
    perc = missing / n_rows
    print(name)
    print('{pos1} von {pos2} Zeilen enthalten fehlende Werte ({pos3:.2%}).\n'.
          format(pos1=missing, pos3=perc, pos2=n_rows))

Anzahl der fehlenden Werte in Spalte "use":
4232 von 671205 Zeilen enthalten fehlende Werte (0.63%).

Anzahl der fehlenden Werte in Spalte "country_code":
8 von 671205 Zeilen enthalten fehlende Werte (0.00%).

Anzahl der fehlenden Werte in Spalte "region":
56800 von 671205 Zeilen enthalten fehlende Werte (8.46%).

Anzahl der fehlenden Werte in Spalte "borrower_genders":
4221 von 671205 Zeilen enthalten fehlende Werte (0.63%).



#### Reminder: reale 0-Werte

Wir erinnern uns an die 0-Werte, welche wir in der statistischen Übersicht gefunden haben. Diese waren in den Spalten **funded_amount** sowie **lender_count** ersichtlich. Mit nachfolgender Methode prüfen wir, ob die Daten vielleich zusammen gehören. Der Gedanke ist der, das wenn die Anzahl der Zeilen beider Spalten übereinstimmt und auch die der Index bei beiden Spalten der gleiche ist, dann gibt es hier folgenden Zusammenhang:

Wo kein Investor, da kein Geld, welches investiert werden kann.

In [24]:
funded_zeros = df.loc[:, 'funded_amount'] == 0
lender_zeros = df.loc[:, 'lender_count'] == 0

n_rows_funded = len(df.loc[funded_zeros])
n_rows_lender = len(df.loc[lender_zeros])

indicies_funded = df.loc[funded_zeros].index
indicies_lender = df.loc[lender_zeros].index

matching_indicies = len(indicies_funded.intersection(indicies_lender))

In [25]:
if (n_rows_funded == n_rows_lender) & (n_rows_funded == matching_indicies):
    print('{} Zeilen betroffen'.format(n_rows_funded))
    print('Anzahl der Zeilen und Indizes passen zusammen!\n')
    print('Die Datensätze haben etwas miteinander zu tun!')

3383 Zeilen betroffen
Anzahl der Zeilen und Indizes passen zusammen!

Die Datensätze haben etwas miteinander zu tun!


Die Vermutung war richtig und wir werden uns diesen Sachverhalt später in der explorativen Datenanalyse genauer anschauen. Zuvor werden wir aber noch die verbliebenden fehlenden Werte bearbeiten müssen.

In [26]:
cols_with_nulls

use                  4232
country_code            8
region              56800
borrower_genders     4221
dtype: int64

Wir fangen mal klein an und schauen uns die fehlenden Werte beim Ländercode an:

#### Fehlende Werte in Spalte 'country_code'

In [27]:
null_values_countryCode = df.loc[:, cols_with_nulls.index[1]].isnull()
df.loc[null_values_countryCode == True, 'country']

310649    Namibia
310650    Namibia
310651    Namibia
310652    Namibia
310653    Namibia
310654    Namibia
310655    Namibia
310656    Namibia
Name: country, dtype: object

Ok, hier fehlt offensichtlich der Code für Namibia. Zuvor können wir nachschauen, ob es wirklich alle Datensätze aus diesem Land sind und filtern nach der Spalte **country**

In [28]:
df.loc[df.loc[:, 'country'] == 'Namibia', 'country'].count()

8

Hier fehlt wirklich nur der Ländercode. Wir suchen diesen heraus, und tragen in entsprechend der ISO 3166 ein.

https://de.wikipedia.org/wiki/ISO-3166-1-Kodierliste#N

In [29]:
df.loc[df.loc[:, 'country'] == 'Namibia', 'country_code'] = 'NA'

In [30]:
# Check ob die Wertzuweisung funktioniert hat
df.loc[df.loc[:, 'country'] == 'Namibia', 'country_code']

310649    NA
310650    NA
310651    NA
310652    NA
310653    NA
310654    NA
310655    NA
310656    NA
Name: country_code, dtype: object

#### Fehlende Werte in Spalte 'use'

In [31]:
null_values_use = df.loc[:, cols_with_nulls.index[0]].isnull()
df_use = df.loc[null_values_use == True]
df_use

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
100,1925.0,1925.0,Agriculture,Agriculture,NaN,AL,Albania,NaN,ALL,39.0,57,NaN,monthly
192,975.0,975.0,Personal Housing Expenses,Housing,NaN,AL,Albania,NaN,ALL,27.0,38,NaN,monthly
255,200.0,775.0,Used Clothing,Clothing,NaN,AL,Albania,NaN,ALL,19.0,7,NaN,monthly
312,900.0,900.0,Decorations Sales,Retail,NaN,AL,Albania,NaN,ALL,14.0,31,NaN,monthly
619,1475.0,1475.0,Dairy,Agriculture,NaN,AL,Albania,NaN,ALL,27.0,55,NaN,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...
670842,500.0,500.0,Poultry,Agriculture,NaN,ZW,Zimbabwe,NaN,USD,24.0,19,NaN,irregular
670917,2950.0,2950.0,Manufacturing,Manufacturing,NaN,ZW,Zimbabwe,NaN,USD,15.0,87,NaN,irregular
670930,800.0,800.0,Used Clothing,Clothing,NaN,ZW,Zimbabwe,NaN,USD,14.0,24,NaN,irregular
671132,500.0,500.0,General Store,Retail,NaN,ZW,Zimbabwe,NaN,USD,8.0,17,NaN,irregular


Hier fällt auf, das wir wohl gleich mehrere Spalten mit fehlenden Werten haben. (_use_, _region_ und _genders_). Um eine bessere Bewertung für ein eventuelles Löschen der Daten zu erhalten, schauen wir uns die anderen Spalten etwas genauer an.

In [32]:
df_use.describe()

,funded_amount,loan_amount,term_in_months,lender_count
count,4232.000000,4232.000000,4232.000000,4232.000000
mean,921.908081,1177.055766,15.126181,26.870983
std,1453.171843,2319.892785,8.912859,40.064176
min,0.000000,25.000000,2.000000,0.000000
25%,250.000000,325.000000,9.000000,7.000000
50%,500.000000,625.000000,14.000000,16.000000
75%,1050.000000,1200.000000,18.000000,33.000000
max,50000.000000,50000.000000,133.000000,1310.000000


In [33]:
df_use.describe().columns

Index(['funded_amount', 'loan_amount', 'term_in_months', 'lender_count'], dtype='object')

In [34]:
df_use.isnull().sum()

funded_amount            0
loan_amount              0
activity                 0
sector                   0
use                   4232
country_code             0
country                  0
region                4227
currency                 0
term_in_months           0
lender_count             0
borrower_genders      4221
repayment_interval       0
dtype: int64

In [35]:
df_use.nunique()

funded_amount         245
loan_amount           238
activity              139
sector                 15
use                     0
country_code           69
country                69
region                  5
currency               53
term_in_months         55
lender_count          192
borrower_genders        5
repayment_interval      3
dtype: int64

In [36]:
for col in df_use.describe().columns:
    sum_col = df.loc[df.loc[:, col] == 0, col].count()
    if sum_col > 0:
        print('Summe der fehlenden Werte in Spalte {col}: {sum}'.format(
            col=col, sum=sum_col))
        perc = sum_col / cols_with_nulls[0]
        print('Dies entspricht {perc:.1%} der bisher nicht finanzierten Projekte\n'.format(perc=perc))

Summe der fehlenden Werte in Spalte funded_amount: 3383
Dies entspricht 79.9% der bisher nicht finanzierten Projekte

Summe der fehlenden Werte in Spalte lender_count: 3383
Dies entspricht 79.9% der bisher nicht finanzierten Projekte



##### Zusammenfassung für die Spalte 'use'

Wir haben hier also einen großen Teil der Daten vor uns, welche wie bereits bei der Untersuchung der 0-Werte gefunden haben. Somit halten wir fest, das wir später nur die Projekte anschauen, welche bisher auch finanziert wurden. Wir notieren aber auch die Anzahl derer, die noch keinen Investor anziehen konnten.

Da wir später auch wissen wollen, wofür die Investoren ihr Geld hergeben, muss zumindest eine kurze Beschreibung des Nutzungszweckes angegeben sein. Daher löschen wir die Datensätze ohne zusätzliche Beschreibung. Als positiver Nebeneffekt entledigen wir uns auch der Daten, welche keine Geschlechterangabe beinhalten.

In [37]:
not_yet_invested = n_rows_funded
missing_description = len(df_use.index)

overlaps = len(indicies_funded.intersection(df_use.index))
indicies_to_drop = indicies_funded.union(df_use.index)
perc = len(indicies_to_drop) / df.shape[0]

print('Projekte, in die noch nicht inverstiert wurde: {proj}'.format(proj=not_yet_invested))
print('Projekte mit fehlendem Verwendungszweck: {miss}'.format(miss=missing_description))
print('Anzahl der sich überschneidenden Datensätze: {over}\n'.format(over=overlaps))


print('Datensätze, welche gelöscht werden {to_drop}'.format(to_drop=len(indicies_to_drop)))
print('Dies entspricht einem Prozentsatz von {perc:.1%}'.format(perc=perc))

Projekte, in die noch nicht inverstiert wurde: 3383
Projekte mit fehlendem Verwendungszweck: 4232
Anzahl der sich überschneidenden Datensätze: 131

Datensätze, welche gelöscht werden 7484
Dies entspricht einem Prozentsatz von 1.1%


Bevor wir diese Daten jedoch löschen, sichern wir uns diese als separaten Datensatz. Wir wollen später noch prüfen, warum zB Investoren ihr Geld in Projekte investieren, obwohl sie den eigentlichen Zweck (den 'use') nicht kennen. Ebenso verfahren wir mit den Projekten, welche keine Investoren anziehen konnten. Auch wenn es gerade einmal ca. 1% unserer bisher gesammelten Daten entspricht, wollen wir uns diese Daten anschauen, um besser beurteilen zu können, ob bestimmte Kategorien besser beworben werden können.

In [38]:
# Sichern der Daten in einen separaten Datensatz
df_check = df.loc[indicies_to_drop]

In [39]:
# löschen der Datensätze
df.drop(index=indicies_to_drop, inplace=True)

In [40]:
# Check, ob das Löschen funktioniert hat
df.isnull().sum()

funded_amount             0
loan_amount               0
activity                  0
sector                    0
use                       0
country_code              0
country                   0
region                51621
currency                  0
term_in_months            0
lender_count              0
borrower_genders          0
repayment_interval        0
dtype: int64

#### Fehlende Werte in Spalte 'borrower_genders'

Die fehlenden Werte in dieser Spalte haben wir bereits im vorherigen Schritt mit eleminiert.

#### Fehlende Werte in Spalte 'region'

In dieser Spalte haben wir die meisten fehlenden Werte mit knapp 8,5% unserer Gesamtdatenmenge. Auch hier schauen wir wieder etwas genauer hin, um keine wichtigen Datensätze zu entfernen.

Generell wäre es ideal, dass wir eine möglichst feinkörnige Granularität der Projekte bezüglich ihres Standortes erhalten. Da uns eine Spalte mit Städten fehlt, wäre die Region zumindest das detailliertes, was wir zur Verfügung haben. Daher überprüfen wir, ob nicht wieder irgendwelche Zusammenhänge erkennbar sind.

In [41]:
df_region = df.loc[df.loc[:, 'region'].isnull() == True]
df_region

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
9,925.0,925.0,Livestock,Agriculture,to build a roof for her animal stable.,AL,Albania,NaN,ALL,26.0,33,female,monthly
17,1275.0,2575.0,Retail,Retail,"to purchase drapes, sheets and blankets to sell.",AL,Albania,NaN,ALL,38.0,24,male,monthly
21,925.0,925.0,Personal Housing Expenses,Housing,To make minor repairs to their washroom and bu...,AL,Albania,NaN,ALL,18.0,31,female,monthly
22,900.0,900.0,Higher education costs,Education,to pay her son's university tuition fees.,AL,Albania,NaN,ALL,22.0,31,female,monthly
24,1450.0,1450.0,Personal Housing Expenses,Housing,to make home improvements by installing energy...,AL,Albania,NaN,ALL,26.0,53,female,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...
653218,2400.0,2400.0,Services,Services,"replace an outdated computer, retain a busines...",US,United States,NaN,USD,12.0,25,female,monthly
653219,5000.0,5000.0,Clothing,Clothing,"produce 100 pieces of my master piece, The Per...",US,United States,NaN,USD,24.0,107,female,monthly
653222,4800.0,4800.0,Agriculture,Agriculture,complete projects in a timely manner without b...,US,United States,NaN,USD,30.0,70,male,monthly
661822,1350.0,1350.0,Restaurant,Food,to purchase materials to maintain her small re...,VN,Vietnam,NaN,VND,14.0,42,"female, female, female",monthly


In [42]:
print('{perc:.1%} der noch verbliebenden Daten sind betroffen.'.format(perc=len(df_region)/len(df)))

7.8% der noch verbliebenden Daten sind betroffen.


In [43]:
df_region.describe()

,funded_amount,loan_amount,term_in_months,lender_count
count,51621.000000,51621.000000,51621.000000,51621.000000
mean,1222.389144,1374.877472,13.438930,30.085605
std,1857.113618,1998.741679,7.916555,38.714436
min,5.000000,25.000000,1.000000,1.000000
25%,300.000000,325.000000,8.000000,10.000000
50%,525.000000,600.000000,14.000000,17.000000
75%,1125.000000,1375.000000,16.000000,34.000000
max,50000.000000,50000.000000,120.000000,1465.000000


In [44]:
df_region.nunique()

funded_amount           519
loan_amount             402
activity                153
sector                   15
use                   45726
country_code             40
country                  40
region                    0
currency                 29
term_in_months           50
lender_count            344
borrower_genders       2445
repayment_interval        4
dtype: int64

Schauen wir uns mal die dazugehörigen Länder und Kategorien an.

In [45]:
df_region['sector'].unique()

array(['Agriculture', 'Retail', 'Housing', 'Education', 'Clothing',
       'Health', 'Services', 'Entertainment', 'Personal Use',
       'Transportation', 'Manufacturing', 'Food', 'Construction',
       'Wholesale', 'Arts'], dtype=object)

In [46]:
df_region['country'].unique()

array(['Albania', 'Armenia', 'Azerbaijan', 'Benin', 'Bolivia', 'Burundi',
       'Cameroon', 'Costa Rica', 'Ecuador', 'El Salvador', 'Guam',
       'Guatemala', 'Haiti', 'Honduras', 'Indonesia', 'Iraq', 'Israel',
       'Jordan', 'Kenya', 'Kosovo', 'Lebanon', 'Madagascar', 'Malawi',
       'Mali', 'Mexico', 'Nicaragua', 'Pakistan', 'Palestine', 'Peru',
       'Philippines', 'Puerto Rico', 'Rwanda', 'Senegal', 'South Africa',
       'Tajikistan', 'The Democratic Republic of the Congo', 'Turkey',
       'United States', 'Vietnam', 'Zimbabwe'], dtype=object)

Es befinden sich größtenteils kleinere Länder, dessen Regionen nicht angegeben wurden. Schauen wir ins das ganze mal im kompletten Datensatz an.

In [47]:
df.groupby(by=['country', 'region']).agg(Angabe_Regionen=('use',
                                                'size')).sort_values(
                                                    ['Angabe_Regionen'],
                                                    ascending=False).head(10)

Angabe_Regionen
country     region                          
Nigeria     Kaduna                      9875
Pakistan    Lahore                      7132
            Rawalpindi                  4455
Peru        Cusco                       3841
Tanzania    Dar es Salaam               3718
Kenya       Kisii                       3545
Philippines Palo, Leyte                 3320
            Narra, Palawan              3197
            Quezon, Palawan             3120
Kenya       Kitale                      3072

Interessante Erkenntnis:
Nigeria und Pakistan sind die Länder, bei denen am häufigsten die Region mit angegeben wurde.

Nun schauen wir uns mal die Top 10 der Länder an, welche am meisten Regionen angegeben haben. Vielleicht generieren wir damit einen gwissen Mehrwert.

In [48]:
top = 10

d_region = {}

name = f'top{top}_regions'

d_region[name] = df.groupby(by=['country']).agg(
    Anzahl_Regionen=('region',
                     'nunique')).sort_values(['Anzahl_Regionen'],
                                             ascending=False).head(top)

perc_regions = d_region[name].sum() / df['region'].nunique()

print(
    '{perc:.1%} der angegebenen Regionen sind auf {best_of} Länder verteilt.'
    .format(perc=perc_regions[0], best_of=10))

d_region[name]

68.5% der angegebenen Regionen sind auf 10 Länder verteilt.


,Anzahl_Regionen
country,
Philippines,3632
Mexico,761
Kyrgyzstan,719
Armenia,666
Mali,627
Peru,577
Georgia,560
Kenya,393
Vietnam,382


##### Zusammenfassung für die Spalte 'region'
Da sich ca. 70% der angegebenen Regionen auf so einen kleinen Anteil der Länder verteilt und wir leider keine anderen nützlichen Daten wie die dazugehörigen Städte oder besser noch Koordinaten haben, ist diese Spalte für uns weniger wertvoll als anfangs gedacht. Daher wird diese Spalte aus dem Datensatz gelöscht.

In [49]:
df.drop(columns='region', inplace=True)

In [50]:
# check ob Löschen funktioniert hat.
df.head()

,funded_amount,loan_amount,activity,sector,use,country_code,country,currency,term_in_months,lender_count,borrower_genders,repayment_interval
0,8000.0,8000.0,Textiles,Arts,"To purchase raw materials, including fabrics, ...",AF,Afghanistan,USD,6.0,308,female,bullet
1,6000.0,6000.0,Textiles,Arts,To buy 9 sewing machines that will increase pr...,AF,Afghanistan,USD,8.0,204,female,bullet
2,825.0,825.0,Agriculture,Agriculture,to buy a dairy cow,AL,Albania,ALL,20.0,26,female,monthly
3,2775.0,2775.0,Agriculture,Agriculture,to repair the damaged animal stable and also d...,AL,Albania,ALL,26.0,101,male,monthly
4,825.0,825.0,Personal Housing Expenses,Housing,to repair windows and doors in her house.,AL,Albania,ALL,14.0,32,female,monthly


In [51]:
df.isnull().sum()

funded_amount         0
loan_amount           0
activity              0
sector                0
use                   0
country_code          0
country               0
currency              0
term_in_months        0
lender_count          0
borrower_genders      0
repayment_interval    0
dtype: int64

### Generierung von Zusatzinformationen

Nachfolgend wollen wir uns den Datensatz noch einmal genauer anschauen und prüfen, ob wir noch zusätzliche Informationen ('_Features_') generieren können. Zum einen bietet die Spalte **borrower_genders** Potenzial aber auch die Spalte der Länder. Hier können wir die Tabelle ebenfalls noch etwas präzisieren.

#### Aufbereitung der Spalte 'borrower_genders'
Die Spalte **borrower_genders** viel bereits schon früher auf. Obwohl es 'nur' eine Spalte für das Geschlecht war, hatten wir bei der statistischen Übersicht bereits festgestellt, das die Werteangabe nicht dem entspricht, was wir uns vorstellen. Daher überprüfen wir diese Spalte nun genauer.

In [52]:
df.nunique()

funded_amount            597
loan_amount              476
activity                 163
sector                    15
use                   422600
country_code              86
country                   86
currency                  67
term_in_months           148
lender_count             498
borrower_genders       11280
repayment_interval         4
dtype: int64

In [53]:
n_genders = df['borrower_genders'].value_counts()
n_genders

female                                                                                                            424653
male                                                                                                              133563
female, female                                                                                                     12118
female, female, female                                                                                             11647
female, female, female, female                                                                                      9018
                                                                                                                   ...  
male, male, male, female, male, male, male, male, male, male, male, male, female, female, female                       1
female, female, female, female, female, female, male, male, male, male, female, male, female, male, male, male         1
male, female, male, male, female

#### Bereinigen/Vorbereiten der Daten
Diese Auflistung scheint sowohl die Anzahl der Investoren als auch dessen jeweiliges Geschlecht zu repräsentieren.
Wir können also die Spalte in entsprechend viele Zusatzspalten aufteilen und die jeweilige Summe der Projektteilnehmer einfügen. Zuvor prüfen wir, wieviele verschiedene Geschlechterbezeichnungen sich in dem Datensatz befinden.

In [54]:
# Wir entfernen erst alle Leerzeichen und teilen die Zeichenkette anschließend am Komma
borrower = df['borrower_genders'].str.replace(' ', '').str.split(',')
borrower

0                                 [female]
1                                 [female]
2                                 [female]
3                                   [male]
4                                 [female]
                        ...               
671200    [female, female, female, female]
671201                            [female]
671202                            [female]
671203    [female, female, female, female]
671204                            [female]
Name: borrower_genders, Length: 663721, dtype: object

Nun ermitteln wir das eindeutige Geschlecht für jeden Datensatz und erzeugen uns hierfür jeweils eine neue Spalte, in denen wir schließlich die jeweilige Anzahl der Projektinitiatoren eintragen können.

https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173

In [55]:
# Funktion, welche die Geschlechterspalte in ein eindimensionales Objekt umwandelt

def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

#to_1D(borrower)

In [56]:
unique_genders = to_1D(borrower).unique()
#unique_genders

Wir haben also nur zwei verschiedene Geschlechter (male + female). Somit können wir jeweile eine Spalte erzeugen und die gezählten Ergebnisse in diese eintragen.

#### Auszählen der Geschlechter je Zeile

In [57]:
d_genders = {}

for gender in unique_genders:
    name = f'borrower_{gender}s'
    d_genders[name] = borrower.apply(lambda lst: (lst.count(gender)))
    d_genders[name].rename(name, inplace=True)
    
#d_genders

In [58]:
d_genders.keys()

dict_keys(['borrower_females', 'borrower_males'])

In [59]:
to_concat = [df]

for key in d_genders.keys():
    to_concat.append(d_genders[key])

#### Zusammenfügen der Datensätze

Nun haben wir sowohl die Spalten, als auch deren Menge. Im nächsten Schritt verbinden wir die einzelnen Spalten mit unserem  ursprünglichen Datensatz und können dann die Projektteilnehmer zuordnen. Danach brauchen wir die Spalte **borrower_genders** nicht mehr und können diese löschen.

In [60]:
df = pd.concat(to_concat, axis=1)
# Option 'copy=False' scheint nicht zu funktionieren
# daher wird über Wertzuweisung der alte Datensatz überschrieben

In [61]:
# check, ob das hinzufügen funktioniert hat
df.head()

,funded_amount,loan_amount,activity,sector,use,country_code,country,currency,term_in_months,lender_count,borrower_genders,repayment_interval,borrower_females,borrower_males
0,8000.0,8000.0,Textiles,Arts,"To purchase raw materials, including fabrics, ...",AF,Afghanistan,USD,6.0,308,female,bullet,1,0
1,6000.0,6000.0,Textiles,Arts,To buy 9 sewing machines that will increase pr...,AF,Afghanistan,USD,8.0,204,female,bullet,1,0
2,825.0,825.0,Agriculture,Agriculture,to buy a dairy cow,AL,Albania,ALL,20.0,26,female,monthly,1,0
3,2775.0,2775.0,Agriculture,Agriculture,to repair the damaged animal stable and also d...,AL,Albania,ALL,26.0,101,male,monthly,0,1
4,825.0,825.0,Personal Housing Expenses,Housing,to repair windows and doors in her house.,AL,Albania,ALL,14.0,32,female,monthly,1,0


In [62]:
# Nun zählen wir noch die Spalten aus und fügen die Summe am Ende der Tabelle wieder an.
df['sum_borrowers'] = df.loc[:, 'borrower_females':].sum(axis=1)

In [63]:
df['sum_borrowers'].nunique()

50

In [64]:
df['sum_borrowers'].describe()

count    663721.000000
mean          2.021601
std           3.419356
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          50.000000
Name: sum_borrowers, dtype: float64

Diese Daten können wir auch gefahrlos kategorisieren. Wir teilen die Personen in Gruppen ein:
 - Einzelpersonen bei n= 1
 - kleine Gruppen bei  1 < n <= 5
 - mittlere Gruppen bei 5 < n <=10
 - große Gruppen n > 10 

In [65]:
group_cat = [
    'Einzelperson', 'kleine Gruppe', 'mittlere Gruppe', 'große Gruppe'
]
group_class = [0, df['sum_borrowers'].min(), 5, 10, df['sum_borrowers'].max()]

df['borrower_groupsize'] = pd.cut(df['sum_borrowers'],
                                  bins=group_class,
                                  labels=group_cat)
df['borrower_groupsize']

0          Einzelperson
1          Einzelperson
2          Einzelperson
3          Einzelperson
4          Einzelperson
              ...      
671200    kleine Gruppe
671201     Einzelperson
671202     Einzelperson
671203    kleine Gruppe
671204     Einzelperson
Name: borrower_groupsize, Length: 663721, dtype: category
Categories (4, object): ['Einzelperson' < 'kleine Gruppe' < 'mittlere Gruppe' < 'große Gruppe']

#### Löschen nicht benötigter Spalten

In [66]:
df.drop(columns='borrower_genders', inplace=True)

In [67]:
# Check, ob Löschen der Zeile geklappt hat
df.head()

,funded_amount,loan_amount,activity,sector,use,country_code,country,currency,term_in_months,lender_count,repayment_interval,borrower_females,borrower_males,sum_borrowers,borrower_groupsize
0,8000.0,8000.0,Textiles,Arts,"To purchase raw materials, including fabrics, ...",AF,Afghanistan,USD,6.0,308,bullet,1,0,1,Einzelperson
1,6000.0,6000.0,Textiles,Arts,To buy 9 sewing machines that will increase pr...,AF,Afghanistan,USD,8.0,204,bullet,1,0,1,Einzelperson
2,825.0,825.0,Agriculture,Agriculture,to buy a dairy cow,AL,Albania,ALL,20.0,26,monthly,1,0,1,Einzelperson
3,2775.0,2775.0,Agriculture,Agriculture,to repair the damaged animal stable and also d...,AL,Albania,ALL,26.0,101,monthly,0,1,1,Einzelperson
4,825.0,825.0,Personal Housing Expenses,Housing,to repair windows and doors in her house.,AL,Albania,ALL,14.0,32,monthly,1,0,1,Einzelperson


#### Aufbereitung der Spalte 'country_code'

Hinter dem Ländercode verbergen sehr viele zusätzliche, zum Teil wirklich hilfreiche informationen. Jedes Land folgende , für uns wichtige Informationen:
 - einen eigenen Schwerpunkt (siehe auch https://de.wikipedia.org/wiki/Mittelpunkte_Deutschlands)
 - eine Bevölkerungszahl
 - BIP
 - Währung (mit Umrechnugskurs zu einer Referenzwährung) --> https://pypi.org/project/CurrencyConverter/
 
Da unser Unternehmen seinen Firmensitz in Deutschland hat, genauer gesagt im wunderschönen Stralsund, ist unsere Referenzwährung der Euro.

#### Umwandlung der Ländercodes

In [68]:
def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return cn_continent

In [69]:
df['continent'] = df['country'].apply(get_continent)

In [70]:
df['continent']

0         AS
1         AS
2         EU
3         EU
4         EU
          ..
671200    AF
671201    AF
671202    AF
671203    AF
671204    AF
Name: continent, Length: 663721, dtype: object

In [71]:
# Prüfung der Ergebnisse
df['continent'].unique()

array(['AS', 'EU', 'NA', 'AF', 'SA', 'Unknown', 'OC'], dtype=object)

Wir habel also einige Länder dabei gehabt, die nicht von der Bibliothek abgedeck sind. Schauen wir uns diese einmal an.

In [72]:
missing_continent = df.loc[df.loc[:, 'continent']=='Unknown', ['country', 'country_code', 'continent']]

In [73]:
missing_continent.groupby(['country']).count()

,country_code,continent
country,,
Cote D'Ivoire,1,1
Kosovo,1398,1398
Myanmar (Burma),1818,1818
The Democratic Republic of the Congo,3067,3067
Timor-Leste,2676,2676


In [74]:
missing_continent['country_code'].unique()

array(['CI', 'XK', 'MM', 'CD', 'TL'], dtype=object)

Das sind zum glück nicht viele Länder, diese können wir händisch eintragen. Welche Kontinente wir setzen müssen, können wir hier entnehmen:

https://gist.github.com/alyssaq/3415363a18610d22f0d307bcaac857cd

In [75]:
continents_to_set = dict.fromkeys(missing_continent['country'].unique())

In [76]:
continents_to_set["Cote D'Ivoire"] = 'AF'
continents_to_set['Kosovo'] = 'EU'
continents_to_set['Myanmar (Burma)'] = 'AS'
continents_to_set['The Democratic Republic of the Congo'] = 'AF'
continents_to_set['Timor-Leste'] = 'AS'

continents_to_set

{"Cote D'Ivoire": 'AF',
 'Kosovo': 'EU',
 'Myanmar (Burma)': 'AS',
 'The Democratic Republic of the Congo': 'AF',
 'Timor-Leste': 'AS'}

Nun weisen wir den jeweiligen Ländern ihre Kontinente zu:

In [77]:
#test = df.loc[:, 'country'] == "Cote D'Ivoire"
#df.loc[test == True, ['continent']]

In [78]:
continents_to_set.keys()

dict_keys(["Cote D'Ivoire", 'Kosovo', 'Myanmar (Burma)', 'The Democratic Republic of the Congo', 'Timor-Leste'])

In [79]:
for key in continents_to_set:
    #print(key)
    country_to_set = df.loc[:, 'country'] == key
    df.loc[country_to_set == True, ['continent']] = continents_to_set[key]

In [80]:
#Check ob es funktioniert hat
df['continent'].unique()

array(['AS', 'EU', 'NA', 'AF', 'SA', 'OC'], dtype=object)

Da wir später die Kontinete als deutsche Langversion haben wollen, mappen wir die Iso-Kurzbezeichnungen entsprechend und fügen diese als neue Spalte hinzu.

In [81]:
continent_mapping = {
    'AS': 'Asien',
    'EU': 'Europa',
    'NA': 'Nordamerika',
    'AF': 'Afrika',
    'SA': 'Südamerika',
    'OC': 'Australien/Ozeanien'
}

In [82]:
for key in continent_mapping:
    #print(key)
    country_to_set = df.loc[:, 'continent'] == key
    df.loc[country_to_set == True, ['continent_long']] = continent_mapping[key]

In [83]:
df['continent_long'].unique()

array(['Asien', 'Europa', 'Nordamerika', 'Afrika', 'Südamerika',
       'Australien/Ozeanien'], dtype=object)

#### Konvertierung der Währung

In [84]:
# Test des Umrechnungskures Dollar --> Euro
c = CurrencyConverter()

print(c.convert(100, 'USD', 'EUR'))

# Standardwährung ist Euro
print(c.convert(100, 'USD'))

94.62528387585164
94.62528387585164


Mit diesem Wissen können wir nun den jeweiligen Umrechnungskurs berechnen. Damit die Daten sich nicht ändern, nehmen wir den Umrechnungskurs vom jeweils letztmöglichen Eintrag.

In [85]:
first_date, last_date = c.bounds['USD']
first_date, last_date

(datetime.date(1999, 1, 4), datetime.date(2022, 5, 5))

In [86]:
c.convert(100, 'USD', date=last_date)

94.62528387585164

Auch müssen wir Prüfen, ob wir in unserem Datensatz keine Währungen haben, die nicht unterstützt werden. Die unterstützen Währungen können direkt aufgerufen werden.

In [87]:
c.currencies

{'AUD',
 'BGN',
 'BRL',
 'CAD',
 'CHF',
 'CNY',
 'CYP',
 'CZK',
 'DKK',
 'EEK',
 'EUR',
 'GBP',
 'HKD',
 'HRK',
 'HUF',
 'IDR',
 'ILS',
 'INR',
 'ISK',
 'JPY',
 'KRW',
 'LTL',
 'LVL',
 'MTL',
 'MXN',
 'MYR',
 'NOK',
 'NZD',
 'PHP',
 'PLN',
 'ROL',
 'RON',
 'RUB',
 'SEK',
 'SGD',
 'SIT',
 'SKK',
 'THB',
 'TRL',
 'TRY',
 'USD',
 'ZAR'}

In [88]:
# Check, ob unser Datensatz nicht unterstützte Währungen enthält
df['currency'].unique()

array(['USD', 'ALL', 'AMD', 'AZN', 'BZD', 'XOF', 'BOB', 'BRL', 'BIF',
       'KHR', 'XAF', 'CLP', 'CNY', 'COP', 'CRC', 'DOP', 'EGP', 'GEL',
       'GHS', 'GTQ', 'HTG', 'HNL', 'INR', 'IDR', 'ILS', 'JOD', 'KES',
       'EUR', 'KGS', 'LAK', 'LBP', 'LSL', 'LRD', 'MGA', 'MWK', 'MXN',
       'MDL', 'MNT', 'MZN', 'MMK', 'NAD', 'NPR', 'NIO', 'NGN', 'PKR',
       'PYG', 'PEN', 'PHP', 'RWF', 'XCD', 'WST', 'SLL', 'SBD', 'ZAR',
       'SSP', 'SRD', 'TJS', 'TZS', 'THB', 'TRY', 'UGX', 'UAH', 'VUV',
       'VND', 'YER', 'ZMW', 'ZWD'], dtype=object)

In [89]:
'USD' in c.currencies

True

In [90]:
unsupported_currencies = df['currency'].apply(lambda x: x not in c.currencies)

In [91]:
df.loc[unsupported_currencies, 'country'].unique()

array(['Albania', 'Armenia', 'Azerbaijan', 'Belize', 'Benin', 'Bolivia',
       'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Chile',
       'Colombia', 'Congo', 'Costa Rica', 'Dominican Republic', 'Egypt',
       'Georgia', 'Ghana', 'Guatemala', 'Haiti', 'Honduras', 'Jordan',
       'Kenya', 'Kyrgyzstan', "Lao People's Democratic Republic",
       'Lebanon', 'Lesotho', 'Liberia', 'Madagascar', 'Malawi', 'Mali',
       'Moldova', 'Mongolia', 'Mozambique', 'Myanmar (Burma)', 'Namibia',
       'Nepal', 'Nicaragua', 'Nigeria', 'Pakistan', 'Palestine',
       'Paraguay', 'Peru', 'Rwanda', 'Saint Vincent and the Grenadines',
       'Samoa', 'Senegal', 'Sierra Leone', 'Solomon Islands',
       'South Sudan', 'Suriname', 'Tajikistan', 'Tanzania', 'Togo',
       'Uganda', 'Ukraine', 'Vanuatu', 'Vietnam', 'Yemen', 'Zambia',
       'Zimbabwe'], dtype=object)

##### Hinweis in eigener Sache:

Ein nachträglicher Blick in die Aufgabenstellung ergab, das der Kredit nur in die jeweilige Landeswährung ausgezahlt wurde. Gesammelt wurde das Geld in US-Dollar. Somit können wir, da unser Firmensitz weiterhin in Deutschland ist, die Währungen in Euro umwandeln. Der Einfachheithalber wird die Summe auf ganze Euro's gerundet.

In [92]:
df['funded_amount_euro'] = df['funded_amount'].apply(
    lambda money_to_convert: round(c.convert(money_to_convert, 'USD', date=last_date), 0))

In [93]:
df['loan_amount_euro'] = df['loan_amount'].apply(
    lambda money_to_convert: round(c.convert(money_to_convert, 'USD', date=last_date), 0))

#### Prozentualer Anteil der finanzierten Projekte

Später wollen wir wissen, wieviele Projekte komplett finanziert wurden und wie hoch der anteil derer ist, die es nicht ganz geschafft haben und wie groß der Fehlbetrag in Prozent ist. Daher erstellen wir eine neue Zusatzspalte, in der wir nach folgender Formel den Anteil errechnen:

\begin{equation}
perc_{recieved} = {\frac {funded_{amount}}{loan_{amount}}}
\end{equation}

In [94]:
df['perc_recieved'] = round(df['funded_amount'] / df['loan_amount'] * 100,0)

In [95]:
# Check ob es funktioniert hat
df['perc_recieved']

0         100.0
1         100.0
2         100.0
3         100.0
4         100.0
          ...  
671200    100.0
671201    100.0
671202    100.0
671203    100.0
671204    100.0
Name: perc_recieved, Length: 663721, dtype: float64

In [96]:
df['perc_recieved'].unique()

array([100.,  60.,  50.,  31.,  86.,  91.,  52.,  37.,  67.,  35.,  47.,
        54.,  71.,   6.,  74.,  68.,  93.,  72.,  53.,  16.,  27.,  95.,
        41.,  63.,  15.,  36.,  21.,  87.,  28.,  79.,  82.,  19.,  45.,
        13.,  18.,  57.,  92.,  39.,  80.,  34.,  69.,  66.,   2.,  70.,
        73.,  64.,  29.,  51.,  89.,  65.,  38.,  75.,  83.,  59.,   5.,
        56.,  43.,  78.,  81.,  76.,  42.,  33.,  90.,  58.,  94.,  88.,
        48.,  30.,  85.,  40.,  17.,  49.,  24.,  26.,  55.,  32.,  84.,
        25.,  61.,  77.,  22.,  14.,  62.,  98.,   9.,  46.,  44.,  20.,
        11.,  12.,   7.,   4.,   3.,  96.,  10.,   1.,  23.,   8.,  97.,
       113.,  99.,   0., 106.])

In [97]:
min_recieved = df['perc_recieved'].min()
max_recieved = df['perc_recieved'].max()

print('Maximalwert: {:3}%'.format(max_recieved))
print('Minimalwert: {:3}%'.format(min_recieved))

Maximalwert: 113.0%
Minimalwert: 0.0%


Interessante Erkenntnis. Es gibt also Projekte, in die mehr investiert wurde, als geplant. Dies schauen wir uns in der EDA genauer an. Wir werden die Projekte noch zusätzlich Kategorisieren.

Wir teilen die Werte in 20% Schritte bis 100% ein. Für alle über 100% gilt die Kategorie 'Überfinanziert'

In [98]:
categories = [
    '<= 5%', '10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%',
    '100%', 'Überfinanziert'
]
classification = [0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, max_recieved]

df['funding_status'] = pd.cut(df['perc_recieved'],
                              bins=classification,
                              labels=categories)
df['funding_status']

0         100%
1         100%
2         100%
3         100%
4         100%
          ... 
671200    100%
671201    100%
671202    100%
671203    100%
671204    100%
Name: funding_status, Length: 663721, dtype: category
Categories (12, object): ['<= 5%' < '10%' < '20%' < '30%' ... '80%' < '90%' < '100%' < 'Überfinanziert']

### Optimierung des Speicherverbrauchs

Wir haben bereits am Anfang den Speicherplatz unsere Daten optimiert. Wir wollen aber trotzdem noch einmal schauen, ob es Optimierungsmöglichkeiten gibt. einen Anhaltspunkt, wie wir die Datentypen weiter optimieren können finden wir auf diesen Seiten:

https://pythonspeed.com/articles/pandas-load-less-data/
https://jakevdp.github.io/PythonDataScienceHandbook/02.01-understanding-data-types.html

Zuvor werden wir nur noch die Spalten neu anordnen, damit sie thematisch besser zusammenpassen. Auch übernehmen wir die Spalte 

In [99]:
df.columns

Index(['funded_amount', 'loan_amount', 'activity', 'sector', 'use',
       'country_code', 'country', 'currency', 'term_in_months', 'lender_count',
       'repayment_interval', 'borrower_females', 'borrower_males',
       'sum_borrowers', 'borrower_groupsize', 'continent', 'continent_long',
       'funded_amount_euro', 'loan_amount_euro', 'perc_recieved',
       'funding_status'],
      dtype='object')

In [100]:
df_final = df[[
    'funded_amount_euro', 'loan_amount_euro', 'perc_recieved', 'funding_status', 'activity',
    'sector', 'use', 'country_code', 'continent', 'continent_long', 'country', 'currency', 'term_in_months',
    'lender_count', 'repayment_interval', 'borrower_females', 'borrower_males',
    'sum_borrowers', 'borrower_groupsize'
]]

#### Check ob Datentypen passen

In [101]:
df_final.dtypes

funded_amount_euro     float64
loan_amount_euro       float64
perc_recieved          float64
funding_status        category
activity                object
sector                  object
use                     object
country_code            object
continent               object
continent_long          object
country                 object
currency                object
term_in_months         float64
lender_count             int64
repayment_interval      object
borrower_females         int64
borrower_males           int64
sum_borrowers            int64
borrower_groupsize    category
dtype: object

Eine erste Prüfung des Datensatzes zeigt, das wir die Spalten **borrower_genders** und **repayment_interval** möglicherweise kategorisieren können. Hierzu müssen wir aber noch die benutzen Einträge prüfen. Des weiteren wird geprüft, ob wir die Spalten **funded_amount**, **loan_amount** und **term_in_months** in Ganzzahlen umwandeln können. Falls nicht, ist dies ein Hinweis für eventuell fehlende Werte. Desweiteren können wir auf diese Weise schon eine Speicherplatzoptimierung erreichen.

In [102]:
mem_opti_pre = my_memory_usage(df_final)
mem_opti_pre # verwendeter Speicherplatz nach Bearbeitung des Datensatzes

genutzer Speicherplatz [MiB]:


445.1

In [103]:
df_final.info(verbose=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 663721 entries, 0 to 671204
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   funded_amount_euro  663721 non-null  float64 
 1   loan_amount_euro    663721 non-null  float64 
 2   perc_recieved       663721 non-null  float64 
 3   funding_status      663655 non-null  category
 4   activity            663721 non-null  object  
 5   sector              663721 non-null  object  
 6   use                 663721 non-null  object  
 7   country_code        663721 non-null  object  
 8   continent           663721 non-null  object  
 9   continent_long      663721 non-null  object  
 10  country             663721 non-null  object  
 11  currency            663721 non-null  object  
 12  term_in_months      663721 non-null  float64 
 13  lender_count        663721 non-null  int64   
 14  repayment_interval  663721 non-null  object  
 15  borrower_females 

In [104]:
cols_to_check = []

unique_limit = 25

for col in df_final.columns:
    n_uniques = df_final[col].nunique()
    if n_uniques < unique_limit:
        print(Fore.GREEN +
              'In Spalte {col} befinden sich {n} eindeutige Werte\n'.format(
                  col=col, n=n_uniques))
        cols_to_check.append(col)
    else:
        print(Fore.RED +
              'Die Spalte {col} beinhaltet mehr als {n} eindeutige Werte\n'.
              format(col=col, n=unique_limit))

Die Spalte funded_amount_euro beinhaltet mehr als 25 eindeutige Werte

Die Spalte loan_amount_euro beinhaltet mehr als 25 eindeutige Werte

Die Spalte perc_recieved beinhaltet mehr als 25 eindeutige Werte

In Spalte funding_status befinden sich 12 eindeutige Werte

Die Spalte activity beinhaltet mehr als 25 eindeutige Werte

In Spalte sector befinden sich 15 eindeutige Werte

Die Spalte use beinhaltet mehr als 25 eindeutige Werte

Die Spalte country_code beinhaltet mehr als 25 eindeutige Werte

In Spalte continent befinden sich 6 eindeutige Werte

In Spalte continent_long befinden sich 6 eindeutige Werte

Die Spalte country beinhaltet mehr als 25 eindeutige Werte

Die Spalte currency beinhaltet mehr als 25 eindeutige Werte

Die Spalte term_in_months beinhaltet mehr als 25 eindeutige Werte

Die Spalte lender_count beinhaltet mehr als 25 eindeutige Werte

In Spalte repayment_interval befinden sich 4 eindeutige Werte

Die Spalte borrower_females beinhaltet mehr als 25 eindeutige Werte

Di

Schauen wir uns die Ergebnisse für **repayment_interval** und **sector** einmal genauer an. Die Spalte **perc_recieved** haben wir schon zuvor kategorisiert.

In [105]:
cols_to_check

['funding_status',
 'sector',
 'continent',
 'continent_long',
 'repayment_interval',
 'borrower_groupsize']

Wir können auch gefahrlos die Länder oder deren Ländercode kategorisieren. Da wir bei der Speicherplatzeinsparung über die Ländernamen prozentual mehr optimieren können, wählen wir noch diese Spalte zusätzlich aus.

In [106]:
cols_to_check.append('country')

In [107]:
for col in cols_to_check:
    print(col + ':')
    print(df_final[col].unique())
    print()

funding_status:
['100%', '60%', '50%', '40%', '90%', ..., '20%', '30%', '<= 5%', 'Überfinanziert', NaN]
Length: 13
Categories (12, object): ['<= 5%' < '10%' < '20%' < '30%' ... '80%' < '90%' < '100%' < 'Überfinanziert']

sector:
['Arts' 'Agriculture' 'Housing' 'Health' 'Construction' 'Food' 'Retail'
 'Services' 'Education' 'Transportation' 'Clothing' 'Personal Use'
 'Manufacturing' 'Entertainment' 'Wholesale']

continent:
['AS' 'EU' 'NA' 'AF' 'SA' 'OC']

continent_long:
['Asien' 'Europa' 'Nordamerika' 'Afrika' 'Südamerika'
 'Australien/Ozeanien']

repayment_interval:
['bullet' 'monthly' 'irregular' 'weekly']

borrower_groupsize:
['Einzelperson', 'kleine Gruppe', 'mittlere Gruppe', 'große Gruppe']
Categories (4, object): ['Einzelperson' < 'kleine Gruppe' < 'mittlere Gruppe' < 'große Gruppe']

country:
['Afghanistan' 'Albania' 'Armenia' 'Azerbaijan' 'Belize' 'Benin' 'Bhutan'
 'Bolivia' 'Brazil' 'Burkina Faso' 'Burundi' 'Cambodia' 'Cameroon' 'Chile'
 'China' 'Colombia' 'Congo' 'Costa Rica

#### Daten kategorisieren

In diesen Spalten können wir die Daten gefahrlos kategorisieren, da die Daten keinerlei fehlende Werte enthalten und im Vergleich zur Anzahl der Spalten nur sehr wenige Werte annehmen können.

In [108]:
for col in cols_to_check:
    if df_final[col].dtype.name == 'category':
        print('Spalte "{col}" hat bereits diesen Datentyp und wird nicht umgewandelt.\n'.format(col=col))
    else:
        df_final.loc[:, col] = df_final.loc[:, col].astype('category')
        #df_final.loc[:, col] = pd.Categorical(df_final[col])
        #df_final[col] = pd.Categorical(df_final[col])
        print('Spalte "{col}" kategorisiert.\n'.format(col=col))

Spalte "funding_status" hat bereits diesen Datentyp und wird nicht umgewandelt.



C:\Users\CAD\AppData\Local\Temp\ipykernel_1052\4170922902.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.loc[:, col] = df_final.loc[:, col].astype('category')
C:\Users\CAD\AppData\Local\Temp\ipykernel_1052\4170922902.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.loc[:, col] = df_final.loc[:, col].astype('category')
C:\Users\CAD\AppData\Local\Temp\ipykernel_1052\4170922902.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

Spalte "sector" kategorisiert.

Spalte "continent" kategorisiert.

Spalte "continent_long" kategorisiert.

Spalte "repayment_interval" kategorisiert.

Spalte "borrower_groupsize" hat bereits diesen Datentyp und wird nicht umgewandelt.

Spalte "country" kategorisiert.



C:\Users\CAD\AppData\Local\Temp\ipykernel_1052\4170922902.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.loc[:, col] = df_final.loc[:, col].astype('category')
C:\Users\CAD\AppData\Local\Temp\ipykernel_1052\4170922902.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.loc[:, col] = df_final.loc[:, col].astype('category')


In [109]:
mem_opti_cat = my_memory_usage(df_final)
mem_opti_cat # verwendeter Speicherplatz nach Kategorisierung

genutzer Speicherplatz [MiB]:


244.4

In [110]:
df_final.info(verbose=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 663721 entries, 0 to 671204
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   funded_amount_euro  663721 non-null  float64 
 1   loan_amount_euro    663721 non-null  float64 
 2   perc_recieved       663721 non-null  float64 
 3   funding_status      663655 non-null  category
 4   activity            663721 non-null  object  
 5   sector              663721 non-null  category
 6   use                 663721 non-null  object  
 7   country_code        663721 non-null  object  
 8   continent           663721 non-null  category
 9   continent_long      663721 non-null  category
 10  country             663721 non-null  category
 11  currency            663721 non-null  object  
 12  term_in_months      663721 non-null  float64 
 13  lender_count        663721 non-null  int64   
 14  repayment_interval  663721 non-null  category
 15  borrower_females 

#### Daten in Ganzzahlen umwandeln

Nun schauen wir uns die Datensätze an, welche Fließkommazahlen beinhalten. Durch einen einfache Check können wir herausfinden, ob wir nur unnötige Präzision mitschleppen. Wir können hier auch gefahrlos auf INT32 konvertieren, da unsere Ganzzahlwerte deutlich kleiner als 2.147.483.647 sind. Somit lässt sich auch hier eine Speicherplatzoptimierung durchführen.

Wir berechnen den Restwert der jeweiligen Zahl bei einer Division durch 1. Ist dieser Wert 0, können wir diese Zahl in eine Ganzzahl umwandeln (siehe Beispiel)

In [111]:
print(str(251.5 % 1) + ' --> Keine Umwandlung in Ganzzahl möglich!')
print(str(251.0 % 1) + ' --> Umwandlung in Ganzzahl möglich')

0.5 --> Keine Umwandlung in Ganzzahl möglich!
0.0 --> Umwandlung in Ganzzahl möglich


In [112]:
df_final.dtypes

funded_amount_euro     float64
loan_amount_euro       float64
perc_recieved          float64
funding_status        category
activity                object
sector                category
use                     object
country_code            object
continent             category
continent_long        category
country               category
currency                object
term_in_months         float64
lender_count             int64
repayment_interval    category
borrower_females         int64
borrower_males           int64
sum_borrowers            int64
borrower_groupsize    category
dtype: object

In [113]:
# Selektieren der Spalten mit Fließkommazahlen
#cols_with_floats = df_final.dtypes.loc[df_final.dtypes.isin(['float', 'float64']) == True]
cols_with_floats = df_final.select_dtypes(include=['float', 'float64'])
cols_with_floats.columns

Index(['funded_amount_euro', 'loan_amount_euro', 'perc_recieved',
       'term_in_months'],
      dtype='object')

In [114]:
cols_with_floats.head()

,funded_amount_euro,loan_amount_euro,perc_recieved,term_in_months
0,7570.0,7570.0,100.0,6.0
1,5678.0,5678.0,100.0,8.0
2,781.0,781.0,100.0,20.0
3,2626.0,2626.0,100.0,26.0
4,781.0,781.0,100.0,14.0


In [115]:
convert_to_int = []
nulls = []

for col in cols_with_floats.columns:
    check_zeros = df_final[col] % 1 == 0
    check_nan = df_final[col] % 1 != np.nan
    print('Spalte ' + col + ':')
    if df_final.loc[check_zeros == False].index.empty:
        print('Daten lassen sich in Ganzzahlen konvertieren\n')
        convert_to_int.append(col)
        continue
    if df_final.loc[check_nan == True].empty:
        print('NULL-Werte gefunden!')
        nulls.append(col)
        continue
    else:
        print('Daten können nicht konvertiert werden\n')
        continue

Spalte funded_amount_euro:
Daten lassen sich in Ganzzahlen konvertieren

Spalte loan_amount_euro:
Daten lassen sich in Ganzzahlen konvertieren

Spalte perc_recieved:
Daten lassen sich in Ganzzahlen konvertieren

Spalte term_in_months:
Daten lassen sich in Ganzzahlen konvertieren



In [116]:
for col in convert_to_int:
    df_final.loc[:, col] = df_final.loc[:, col].astype(np.int32, copy=False)
    print('Spalte umgewandelt: {col}'.format(col=col))

Spalte umgewandelt: funded_amount_euro
Spalte umgewandelt: loan_amount_euro
Spalte umgewandelt: perc_recieved
Spalte umgewandelt: term_in_months


C:\Users\CAD\AppData\Local\Temp\ipykernel_1052\122533370.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.loc[:, col] = df_final.loc[:, col].astype(np.int32, copy=False)


In [117]:
mem_opti_float2int = my_memory_usage(df_final)
mem_opti_float2int # verwendeter Speicherplatz nach Umwandlung von Fließkommazahlemn in Ganzzahlen

genutzer Speicherplatz [MiB]:


234.3

In [118]:
df_final.info(verbose=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 663721 entries, 0 to 671204
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   funded_amount_euro  663721 non-null  int32   
 1   loan_amount_euro    663721 non-null  int32   
 2   perc_recieved       663721 non-null  int32   
 3   funding_status      663655 non-null  category
 4   activity            663721 non-null  object  
 5   sector              663721 non-null  category
 6   use                 663721 non-null  object  
 7   country_code        663721 non-null  object  
 8   continent           663721 non-null  category
 9   continent_long      663721 non-null  category
 10  country             663721 non-null  category
 11  currency            663721 non-null  object  
 12  term_in_months      663721 non-null  int32   
 13  lender_count        663721 non-null  int64   
 14  repayment_interval  663721 non-null  category
 15  borrower_females 

In [119]:
# Check, ob die Umwandlung funktioniert hat
df_final.dtypes

funded_amount_euro       int32
loan_amount_euro         int32
perc_recieved            int32
funding_status        category
activity                object
sector                category
use                     object
country_code            object
continent             category
continent_long        category
country               category
currency                object
term_in_months           int32
lender_count             int64
repayment_interval    category
borrower_females         int64
borrower_males           int64
sum_borrowers            int64
borrower_groupsize    category
dtype: object

#### Verkleinern der Ganzzahlen in passendere Datentypen

In [120]:
cols_with_ints = df_final.select_dtypes(include=['int32', 'int64'])
cols_with_ints.dtypes

funded_amount_euro    int32
loan_amount_euro      int32
perc_recieved         int32
term_in_months        int32
lender_count          int64
borrower_females      int64
borrower_males        int64
sum_borrowers         int64
dtype: object

In [121]:
for col in cols_with_ints:
    print('{value:>10} --> Maximalwert in Spalte {col}'.format(col=col, value=df[col].max()))

   94625.0 --> Maximalwert in Spalte funded_amount_euro
   94625.0 --> Maximalwert in Spalte loan_amount_euro
     113.0 --> Maximalwert in Spalte perc_recieved
     158.0 --> Maximalwert in Spalte term_in_months
      2986 --> Maximalwert in Spalte lender_count
        50 --> Maximalwert in Spalte borrower_females
        44 --> Maximalwert in Spalte borrower_males
        50 --> Maximalwert in Spalte sum_borrowers


Wir haben hier also die Möglichkeit, in den letzten vier Spalten den Datentyp nochmals zu verringern.
Hierbei gilt zu beachten, das es zwei verschiedene Arten von Ganzzahlen gibt. Die Zahlen, die nur positiv sein können (_unsigned_) und auch negativ (_signed_), im deutschen auch als Vorzeichen bezeichnet..


Folgende Maximalgrößen sind, laut oben verlinkter Seiten, für die Datentypen zulässig:

| int_8bit | int_16bit |int_32bit | int_64bit |
| :---: | :---: | :---: | :---: |
| [$-2^7$, $2^7-1$] | [$-2^{15}$, $2^{15}-1$] | [$-2^{31}$, $2^{31}-1$] |[$-2^{63}$, $2^{63}-1$] |


| uint_8bit | uint_16bit |uint_32bit | uint_64bit |
| :---: | :---: | :---: | :---: |
| $(2^{7}-1)$ | $(2^{15}-1)$ | $(2^{31}-1)$ |$(2^{63}-1)$ |

In [122]:
int_sizes = {}

for i in [1, 2, 4, 8]:
    name = 'uint{}'.format(8 * i)
    #print(name)
    int_sizes[name] = {'max_size': 2*2**(8 * i - 1) - 1, 'columns': []}

int_sizes

{'uint8': {'max_size': 255, 'columns': []},
 'uint16': {'max_size': 65535, 'columns': []},
 'uint32': {'max_size': 4294967295, 'columns': []},
 'uint64': {'max_size': 18446744073709551615, 'columns': []}}

In [123]:
for col in cols_with_ints.columns:
    for key in int_sizes.keys():
        #print(key)
        max_value = int_sizes[key]['max_size']
        if df_final[col].max() <= max_value:
            print(f'Spalte {col}:')
            print(f'vorhandener Maximalwert: {df_final[col].max():,}')
            print(
                f'Der ideale Datentyp für diese Spalte ist {key}.\nerlaubter Maximalwert = {max_value:,} bytes\n'
            )
            df_final.loc[:, col] = df_final.loc[:, col].astype(key, copy=False)
            break

Spalte funded_amount_euro:
vorhandener Maximalwert: 94,625
Der ideale Datentyp für diese Spalte ist uint32.
erlaubter Maximalwert = 4,294,967,295 bytes

Spalte loan_amount_euro:
vorhandener Maximalwert: 94,625
Der ideale Datentyp für diese Spalte ist uint32.
erlaubter Maximalwert = 4,294,967,295 bytes

Spalte perc_recieved:
vorhandener Maximalwert: 113
Der ideale Datentyp für diese Spalte ist uint8.
erlaubter Maximalwert = 255 bytes

Spalte term_in_months:
vorhandener Maximalwert: 158
Der ideale Datentyp für diese Spalte ist uint8.
erlaubter Maximalwert = 255 bytes

Spalte lender_count:
vorhandener Maximalwert: 2,986
Der ideale Datentyp für diese Spalte ist uint16.
erlaubter Maximalwert = 65,535 bytes

Spalte borrower_females:
vorhandener Maximalwert: 50
Der ideale Datentyp für diese Spalte ist uint8.
erlaubter Maximalwert = 255 bytes

Spalte borrower_males:
vorhandener Maximalwert: 44
Der ideale Datentyp für diese Spalte ist uint8.
erlaubter Maximalwert = 255 bytes

Spalte sum_borrowe

C:\Users\CAD\AppData\Local\Temp\ipykernel_1052\2837985473.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.loc[:, col] = df_final.loc[:, col].astype(key, copy=False)


In [124]:
# Check ob umwandlung funktioniert hat
df_final.dtypes

funded_amount_euro      uint32
loan_amount_euro        uint32
perc_recieved            uint8
funding_status        category
activity                object
sector                category
use                     object
country_code            object
continent             category
continent_long        category
country               category
currency                object
term_in_months           uint8
lender_count            uint16
repayment_interval    category
borrower_females         uint8
borrower_males           uint8
sum_borrowers            uint8
borrower_groupsize    category
dtype: object

In [125]:
mem_opti_int = my_memory_usage(df_final)
mem_opti_int # verwendeter Speicherplatz nach Optimierung der Ganzzahlspalten

genutzer Speicherplatz [MiB]:


213.4

In [126]:
df_final.info(verbose=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 663721 entries, 0 to 671204
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   funded_amount_euro  663721 non-null  uint32  
 1   loan_amount_euro    663721 non-null  uint32  
 2   perc_recieved       663721 non-null  uint8   
 3   funding_status      663655 non-null  category
 4   activity            663721 non-null  object  
 5   sector              663721 non-null  category
 6   use                 663721 non-null  object  
 7   country_code        663721 non-null  object  
 8   continent           663721 non-null  category
 9   continent_long      663721 non-null  category
 10  country             663721 non-null  category
 11  currency            663721 non-null  object  
 12  term_in_months      663721 non-null  uint8   
 13  lender_count        663721 non-null  uint16  
 14  repayment_interval  663721 non-null  category
 15  borrower_females 

#### Zusammenfassung

In [127]:
saved_memory1 = 1 - mem_opti_int / mem_usage_init
saved_memory2 = round(mem_usage_init - mem_opti_int, 1)

print('''
Trotz der zusätzlichen Spalten, die wir im Laufe des Preprocessings eingefügt haben,
konnten wir mit den durchgeführten Maßnahmen den Speicherplatzverbrauch um ca. {mem:.0%} reduzieren.

Dies entspricht in absoluten Zahlen etwa {mem_saved} MiB.
      '''.format(mem=saved_memory1, mem_saved=saved_memory2))


Trotz der zusätzlichen Spalten, die wir im Laufe des Preprocessings eingefügt haben,
konnten wir mit den durchgeführten Maßnahmen den Speicherplatzverbrauch um ca. 50% reduzieren.

Dies entspricht in absoluten Zahlen etwa 216.5 MiB.
      


### Sichern der Datensätze

Diesen Stand sichern wir uns nun als 'pickle'-Datei (wecken unseren Datensatz also ein und machen ihn haltbar :D ). Da dieses Dateiformat versionsabhängig ist, wird der Datensatz zusätzlich noch als CSV-Datei gespeichert.

In [128]:
filename = 'SH_CrowdInvesting_preprocessing'

df_final.to_pickle(f'{filename}.pkl')
df_final.to_csv(f'{filename}.csv', index=False, sep=',')

df_check.to_pickle('dataset_to_check.pkl')
df_check.to_csv('dataset_to_check.csv', index=False, sep=',')